How to identify how many Hidden layers to take?

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
data= pd.read_csv('Churn_Modelling.csv')

In [15]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
data= data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender= LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo= OneHotEncoder(handle_unknown='ignore')
geo_encoded= onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data= pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
X= data.drop('Exited',axis=1)
y= data['Exited']

X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=42)
scaler= StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)

In [17]:
import pickle
with open('label_encoder_gender_hyper.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)
with open('onehot_encoder_geo_hyper.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)
with open('scale_hyper.pkl','wb') as f:
    pickle.dump(scaler,f)

In [18]:
def create_model(neurons=32, layers=1, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, activation=activation,input_shape=(X_train.shape[1],)))
    for i in range(layers-1):
        model.add(Dense(neurons, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [22]:
param_grid = {
    'neurons': [16,32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100],
}

In [27]:
## Create a KerasClassifier with layers and neurons parameters
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0, layers=1, neurons=32)

## Create a GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

## Print the results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

/Users/harshkumar/Desktop/Main/Projects/Classification DL ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/harshkumar/Desktop/Main/Projects/Classification DL ANN/venv1/lib/python3.11/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/harshkumar/Desktop/Main/Projects/Classification DL ANN/venv1/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/harshkumar/Desk

Best: 0.8576238547825933 using {'epochs': 100, 'layers': 1, 'neurons': 32}
